# Network utility functions

In [2]:
%load_ext autoreload
%autoreload 2

import os
import shutil

import numpy as np
from scipy.stats import laplace
import matplotlib.pyplot as plt

from src.events.Events import (
    Events,
)
from src.events.tools.generation.pix2nvs import Pix2Eve
from src.events.tools.generation.stimuli_gen import (
    moving_lines,
    disparity_bars
)
from src.events.analysis.event_analysis import (
    quantity_variation
)
from src.frames.frame_analysis import (
    load_frames,
    rectify_frames,
    write_frames,
    stereo_matching,
    extract_frames_video
)
import src.spiking_network.network.network_params as network_params
from src.spiking_network.network.neuvisys import (
    SpikingNetwork,
    delete_files,
    clean_network,
    shuffle_weights,
)
from src.spiking_network.planning.network_planner import (
    create_networks,
    random_params,
    launch_neuvisys_multi_pass,
    change_param,
    divide_visual_field,
)

os.chdir("/home/thomas/neuvisys-analysis/src")
home = "/home/thomas/"

SMALL_SIZE = 20
MEDIUM_SIZE = 25
BIGGER_SIZE = 30

plt.rc('figure', figsize=(30, 8))
plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE,
       labelsize=MEDIUM_SIZE)  # fontsize of the axes title, fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize

# network_path = home + "neuvisys-dv/configuration/network_ref_vh/"
network_path = home + "Desktop/Experiment/network_0/"

rotations = np.array([0, 23, 45, 68, 90, 113, 135, 158, 180, 203, 225, 248, 270, 293, 315, 338])
disparities = np.array([-8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8])

## CAUTION: Delete Weights

In [ ]:
clean_network("/home/thomas/Desktop/Experiment/network_0/", [0, 1])

## Event manipulation:
Functions for loading, writing or creating event files in the different formats (aedat4, npz, rosbag...)

#### Load

In [15]:
# events = Events("/home/thomas/Videos/natural/pavin_aquisitions/pavin.h5")
events = Events(["/home/thomas/Videos/natural/pavin_aquisitions/left.aedat4",
                 "/home/thomas/Videos/natural/pavin_aquisitions/right.aedat4"], [0, 1], width=346, height=260) # stereo

#### Modify

In [30]:
print(divide_visual_field(246, 180, 3, 3, 40, 40))
# print(divide_visual_field(640, 480, 1, 1, 346, 260))

([0.0, 103.0, 206.0], [0.0, 70.0, 140.0])


In [10]:
events.downsample()

In [16]:
events.sort_events()

In [20]:
# events.create_disparity([(0, 0, -2, 2), (115, 0, -2, 1), (230, 0, -1, 1),
#                          (0, 86, -1, 0), (115, 86, 0, 0), (230, 86, 1, 0),
#                          (0, 172, 1, -1), (115, 172, 2, -1), (230, 172, 2, -2)], 115, 86)
events.create_disparity([(0, 0, 2, -2), (115, 0, 2, -1), (230, 0, 1, -1),
                         (0, 86, 1, 0), (115, 86, 0, 0), (230, 86, -1, 0),
                         (0, 172, -1, 1), (115, 172, -2, 1), (230, 172, -2, 2)], 115, 86)

In [17]:
events.rectify_events(-4, 8, 4, -8)

In [19]:
events.rotate(np.radians(180), (346//2, 260//2))

In [28]:
events.crop(0, 130, 346, 130)

In [4]:
events.resize(100, 80)

In [ ]:
events.get_cameras()

#### Save

In [19]:
events.save_as_file("/home/thomas/Videos/natural/pavin_aquisitions/stereo_pavin_2_3.h5")

In [ ]:
vf_network = [[50, 40, 90, 80], [153, 110, 193, 150], [256, 180, 296, 220]]
events.to_video(50, "/home/thomas/Bureau/pavin_rect")

In [ ]:
list_events = []
for rotation in [0, 23, 45, 68, 90, 113, 135, 158, 180, 203, 225, 248, 270, 293, 315, 338]:
    list_events.append(np.load("/home/thomas/Videos/lines/rotations/" + str(rotation) + ".npz"))
# events = concatenate_npz(list_events)

In [ ]:
for file in os.listdir("/home/thomas/Videos/dsec/"):
    print(file, ": \n", "loading left events...")
    events = Events("/home/thomas/Videos/dsec/"+file+"/events/left/events.h5")
    print("loading right events...")
    events.add_events("/home/thomas/Videos/dsec/"+file+"/events/right/events.h5", 1)
    # print("resizing events...")
    # events.crop(147, 110, 346, 260)
    print("sorting events...")
    events.sort_events()
    print("saving events...")
    events.save_as_file("/home/thomas/Videos/DSEC/"+file+".h5")

## Network creation and launch

In [ ]:
# params = {"simple_cell_config" : {"ETA_LTP": [0.0077, 0.0077]}, "complex_cell_config" : {"ETA_LTP": [0.0077, 0.0077]}}
# create_networks(home + "neuvisys-dv/cmake-build-release", home + "Bureau", 2, {})
random_params("/home/thomas/neuvisys-dv/cmake-build-release", "/home/thomas/Desktop/Experiment", 1)

In [ ]:
launch_neuvisys_multi_pass("/home/thomas/neuvisys-dv/cmake-build-release/neuvisys-exe",
                           "/home/thomas/Desktop/Experiment/network_0/configs/network_config.json",
                           "/home/thomas/Desktop/Events/left_e/vlines.npz", 1)

In [ ]:
# Launch training of multiple networks

n_networks = 100
exec_path = home + "neuvisys-dv/cmake-build-release/neuvisys-exe"
networks_path = home + "Desktop/test/"
event_path = home + "Desktop/shapes.npz"

params = {"simple_cell_config": {"TARGET_SPIKE_RATE": [0.1, 0.2, 0.3]}}

create_networks(exec_path, networks_path, n_networks, params)
nb_iterations = 5

df = []

for i in range(0, n_networks):
    launch_neuvisys_multi_pass(
        exec_path, networks_path + "network_" + str(i) + "/configs/network_config.json", event_path, nb_iterations,
    )

    spinet = SpikingNetwork(networks_path + "network_" + str(i) + "/")
    # display_network([spinet], 0)
    # update_dataframe(df, spinet)

# df = pd.DataFrame(df)
# df.to_csv(home + "Desktop/test/df")

In [ ]:
events_path = "/home/thomas/Videos/DSEC/"
list_events = os.listdir(events_path)
exec_path = home + "neuvisys-dv/cmake-build-release/neuvisys-exe"
network_path = home + "Desktop/Experiment/"

create_networks(exec_path, network_path, 1, network_params.disparity_9regions())

for event in list_events:
    launch_neuvisys_multi_pass(exec_path, network_path + "network_0/", events_path + event, 1)

#### Orientation response

In [ ]:
exec_path = home + "neuvisys-dv/cmake-build-release/neuvisys-exe"
network_path = "/home/thomas/Desktop/Networks/lateral_weight_plot/network_0/"
event_rotation = home + "Videos/lines/rotations/"

spikes = [[], []]
for rot in rotations:
    launch_neuvisys_multi_pass(exec_path, network_path + "configs/network_config.json",
                               event_rotation + str(rot) + ".npz", 5)
    spinet = SpikingNetwork(network_path)
    spikes[0].append(spinet.spikes[0])
    spikes[1].append(spinet.spikes[1])
spinet.save_rotation_response(spikes, rotations)

#### Disparity response

In [ ]:
exec_path = home + "neuvisys-dv/cmake-build-release/neuvisys-exe"
network_path = home + "Desktop/network_experiment/"
event_disparity = home + "Videos/disparity/base_disp/"

spikes = [[], []]
for disp in disparities:
    launch_neuvisys_multi_pass(exec_path, network_path + "configs/network_config.json",
                               event_disparity + str(disp) + ".npz", 5)
    spinet = SpikingNetwork(network_path)
    spikes[0].append(spinet.spikes[0])
    spikes[1].append(spinet.spikes[1])
spinet.save_complex_disparities(spikes[1], disparities)

#### Inhibition weight learning

In [ ]:
exec_path = home + "neuvisys-dv/cmake-build-release/neuvisys-exe"
networks_path = home + "Desktop/Experiment/"
event_path = home + "Videos/disparity/"

# orientations
shapes = home + "Videos/shapes.npz"
h_events = home + "Videos/lines/hlines.npz"
v_events = home + "Videos/lines/vlines.npz"
vh_events = home + "Videos/lines/vhlines.npz"
sim_h = home + "Videos/lines/sim_h.npz"
sim_v = home + "Videos/lines/sim_v.npz"
sim_vh = home + "Videos/lines/sim_vh.npz"
rot_events = home + "Videos/lines/rotations.npz"
list_events_rot = [home + "Videos/lines/rotations/" + str(rotation) + ".npz" for rotation in rotations]

# disparity
events_0 = home + "Videos/disparity/disp_forward/0.npz"
list_disparities = [home + "Videos/disparity/disp_forward/" + str(disp) + ".npz" for disp in
                    [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]]

In [ ]:
create_networks(exec_path, networks_path, 1, network_params.disparity_9regions())

##### learning the base network

In [ ]:
## learning the base network
path = networks_path + "network_0/"

# draw = np.random.randint(-5, 6, size=300)
# for ind in draw:
#     launch_neuvisys_multi_pass(exec_path, path + "configs/network_config.json", list_disparities[ind], 1)

# for i in range(100):
#     launch_neuvisys_multi_pass(exec_path, path + "configs/network_config.json", "/home/thomas/Videos/DSEC/car.npz", 1)

for i in range(3):
    launch_neuvisys_multi_pass(exec_path, path + "configs/network_config.json", sim_vh, 5)

##### learning inhibition weights

In [ ]:
## learning inhibition weights
path = networks_path + "network_0/"

draw = np.round(laplace(0, 1.5).rvs(size=200)).astype(np.int64)
rotations_distribution = [list_events_rot[index] for index in draw if 16 > index > -17]

shutil.copytree(path, networks_path + "network_inhibition")
change_param(networks_path + "network_inhibition/configs/simple_cell_config.json", "STDP_LEARNING", "inhibitory")
change_param(networks_path + "network_inhibition/configs/complex_cell_config.json", "STDP_LEARNING", "none")

import re
distribution = ([int(re.findall('[0-9]+', string)[0]) for string in rotations_distribution])
plt.figure()
plt.hist(distribution, bins=np.arange(np.min(distribution) + 1, np.max(distribution) - 1, 22.5), align="left")

In [ ]:
np.save(networks_path + "network_inhibition/distribution", draw)

In [ ]:
# for eventfile in rotations_distribution:
#     launch_neuvisys_multi_pass(exec_path, networks_path + "network_inhibition/configs/network_config.json", eventfile, 1)

# for ind in draw:
#     launch_neuvisys_multi_pass(exec_path, networks_path + "network_inhibition/configs/network_config.json", list_events_rot[int(ind)], 1)

for i in range(10):
    launch_neuvisys_multi_pass(exec_path, networks_path + "network_inhibition/configs/network_config.json", sim_h, 5)

In [ ]:
shutil.copytree(networks_path + "network_inhibition", networks_path + "network_control")
shutil.copytree(networks_path + "network_inhibition", networks_path + "network_experiment")

change_param(networks_path + "network_control/configs/simple_cell_config.json", "STDP_LEARNING", "none")
change_param(networks_path + "network_control/configs/complex_cell_config.json", "STDP_LEARNING", "none")
change_param(networks_path + "network_experiment/configs/simple_cell_config.json", "STDP_LEARNING", "none")
change_param(networks_path + "network_experiment/configs/complex_cell_config.json", "STDP_LEARNING", "none")

##### experiment

In [ ]:
launch_neuvisys_multi_pass(exec_path, networks_path + "network_experiment/configs/network_config.json", sim_vh, 2)

##### control test

In [ ]:
shuffle_weights(networks_path + "network_control/weights/0/")
launch_neuvisys_multi_pass(exec_path, networks_path + "network_control/configs/network_config.json", sim_vh, 2)

## Frame manipulation

In [11]:
frames = load_frames("/home/thomas/Videos/natural/pavin_aquisitions/2/pavin_2_3.aedat4")

In [12]:
rect_frames = rectify_frames(frames, -4, 8, 4, -8)

write_frames(
    "/home/thomas/Bureau/im1/", rect_frames, ([10, 84, 158, 232, 306], [20, 83, 146, 209]),
)

In [ ]:
# Stereo matching
disp_frames, disp_nb_frames = stereo_matching(
    "/home/thomas/Bureau/im1/", [10, 84, 158, 232], [20, 83, 146], range(0, 200),
)

In [ ]:
extract_frames_video(["/home/thomas/Bureau/pavin_rect_0.mp4", "/home/thomas/Bureau/pavin_rect_1.mp4"])

## Create event videos

Generate frames of the desired visual stimuli, then convert those frames into an event stream

In [ ]:
framerate = 1000
time_gap = 1e6 * 1 / framerate
pix2eve = Pix2Eve(
    time_gap=time_gap,
    log_threshold=0,
    map_threshold=0.4,
    n_max=5,
    adapt_thresh_coef_shift=0,
    timestamp_noise=50
)
folder = home + "Desktop/Events/left_f/"

In [ ]:
disparity_bars(folder, disparities=[0, 0, 0, 0], framerate=framerate)
disparity_bars(folder, disparities=[8, 6, 4, 2], framerate=framerate)

In [ ]:
for rotation in [0, 23, 45, 68, 90, 113, 135, 158, 180, 203, 225, 248, 270, 293, 315, 338]:
    os.mkdir(folder + str(rotation) + "/")
    moving_lines(folder + str(rotation) + "/", time=2, rotation=rotation, framerate=framerate)

In [ ]:
moving_lines(folder, time=2, framerate=framerate, rotation=0, disparity=0, frame_start=0)

In [ ]:
print("Converting frames to event -> " + folder)

events = pix2eve.run("/home/thomas/Desktop/Events/left_f/")
events = events[events[:, 0].argsort()]
# write_npz(home + "Desktop/Events/left_e/vlines", events)

In [ ]:
folder = home + "Desktop/Events/"

moving_lines(folder + "left_f/", time=2, framerate=framerate, speed=-200, rotation=0, disparity=0, frame_start=0)
events = pix2eve.run(folder + "left_f/")
events = events[events[:, 0].argsort()]
# write_npz(folder + "/left_e/0", events)

for i in range(-8, 9):
    moving_lines(folder + "right_f/", time=2, framerate=framerate, speed=-200, rotation=0, disparity=i, frame_start=0)
    events = pix2eve.run(folder + "right_f/")
    events = events[events[:, 0].argsort()]
    # write_npz(folder + "/right_e/" + str(i), events)
    delete_files(folder + "right_f/")
    Events(folder + "left_e/0.npz", folder + "right_e/" + str(i) + ".npz").save_file(folder + str(i))